---
description: action list
output-file: functions.html
title: functions

---

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#| default_exp functions

In [3]:
#| export

import importlib.resources
import os
import shutil
import winreg
from pathlib import Path
import re
import win32com.client as win32 
import pythoncom 
from win32com.client import Dispatch
from win32com import client

from hwpapi.constants import char_fields, para_fields

In [4]:
#| export

def get_font_name(text):
    m = re.search("(^.+?)\s[A-Z0-9]+\.HFT", text)
    return m.group(1) if m else None

In [5]:
#| export


def dispatch(app_name):
    """캐시가 충돌하는 문제를 해결하기 위해 실행합니다. 에러가 발생할 경우 기존 캐시를 삭제하고 다시 불러옵니다."""
    try:
        from win32com import client

        app = client.gencache.EnsureDispatch(app_name)
    except AttributeError:
        # Corner case dependencies.
        import os
        import re
        import shutil
        import sys

        # Remove cache and try again.
        MODULE_LIST = [m.__name__ for m in sys.modules.values()]
        for module in MODULE_LIST:
            if re.match(r"win32com\.gen_py\..+", module):
                del sys.modules[module]
        shutil.rmtree(os.path.join(os.environ.get("LOCALAPPDATA"), "Temp", "gen_py"))
        from win32com import client

        app = client.gencache.EnsureDispatch(app_name)
    return app

In [6]:
#| export

def get_hwp_objects():
    context = pythoncom.CreateBindCtx(0)
    
    # 현재 실행중인 프로세스를 가져옵니다. 
    running_coms = pythoncom.GetRunningObjectTable()
    monikers = running_coms.EnumRunning()

    hwp_objects = []
    for moniker in monikers:
        name = moniker.GetDisplayName(context,moniker);
        # moniker의 DisplayName을 통해 한글을 가져옵니다
        # 한글의 경우 HwpObject.버전으로 각 버전별 실행 이름을 설정합니다. 
        if re.match("!HwpObject", name):
            # 120은 한글 2022의 경우입니다. 
            # 현재 moniker를 통해 ROT에서 한글의 object를 가져옵니다. 
            obje = running_coms.GetObject(moniker)
            # 가져온 object를 Dispatch를 통해 사용할수 있는 객체로 변환시킵니다. 
            hwp_objects.append(obje.QueryInterface(pythoncom.IID_IDispatch))
    return hwp_objects

In [7]:
#| export


def get_absolute_path(path):
    """파일 절대 경로를 반환합니다."""
    name = Path(path)
    return name.absolute().as_posix()

In [8]:
#| export


def get_dll_path(package_name, dll_filename):
    """패키지에서 dll 경로를 확보합니다."""
    try:
        with importlib.resources.path(package_name, dll_filename) as dll_path:
            return str(dll_path)
    except FileNotFoundError as e:
        raise FileNotFoundError(
            f"The DLL file '{dll_filename}' was not found in the package '{package_name}'."
        ) from e

In [9]:
#| export


def add_dll_to_registry(dll_path, key_path):
    """레지스트리에 dll을 등록합니다."""
    try:
        # Connect to the registry and open the specified key
        registry_key = winreg.OpenKey(
            winreg.HKEY_CURRENT_USER, key_path, 0, winreg.KEY_SET_VALUE
        )

        # Set the value for the new registry entry as a string (REG_SZ)
        winreg.SetValueEx(
            registry_key, "FilePathCheckerModule", 0, winreg.REG_SZ, dll_path
        )

        # Close the registry key
        winreg.CloseKey(registry_key)
        print("DLL path added to registry as a string value successfully.")
    except WindowsError as e:
        print("Error while adding DLL path to registry: ", e)

In [10]:
#| export


def get_registry_value(key_path, value_name):
    """레지스트리에 값이 있는지 확인해 봅니다."""
    try:
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, key_path) as key:
            value, value_type = winreg.QueryValueEx(key, value_name)
            return value
    except FileNotFoundError:
        return None

In [11]:
#| export


def check_dll(dll_path=None):
    """dll 모듈을 등록합니다."""
    dll_path = dll_path if dll_path else get_dll_path("hwpapi", "FilePathCheckerModuleExample.dll")
    key_path = "SOFTWARE\\HNC\\HwpAutomation\\Modules"
    value_name = "FilePathCheckerModule"

    value = get_registry_value(key_path, value_name)

    if value is None:
        add_dll_to_registry(dll_path, key_path)
    return True

In [12]:
#| export
def get_value(dict_, key):
    """딕셔너리에서 키를 찾아 값을 반환합니다. 반환할 값이 없으면 키에러와 함께 가능한 키를 알려줍니다."""
    if key is None:
        return None
    try:
        return dict_[key]
    except KeyError:
        raise KeyError(
            f"{key}를 해당하는 키 중 찾을 수 없습니다. 키는 {', '.join(dict_.keys())} 중에 있어야 합니다."
        )

In [13]:
#| export
def get_key(dict_, value):
    """딕셔너리에서 값를 찾아 키를 반환합니다. 반환할 값이 없으면 키에러와 함께 가능한 키를 알려줍니다."""
    if value is None:
        return None
    try:
        return dict([(v, k) for k, v in dict_.items()])[value]
    except KeyError:
        raise KeyError(
            f"{value}를 해당하는 키 중 찾을 수 없습니다. 키는 {', '.join(dict_.values())} 중에 있어야 합니다."
        )

In [14]:
#| export 

def convert2int(_dict, value):
    if value is None:
        return value
    if isinstance(value, str):
        return get_value(_dict, value)
    if isinstance(value, int):
        return value
    if isinstance(value, float):
        return int(value)

In [15]:
#| export 

def set_pset(p, value_dict:dict):
    for field in dir(p):
        value = value_dict.get(field, None)
        if value is None:
            continue
        setattr(p, field, value)

    return p

In [16]:
#| export

def get_charshape_pset(p):
    return {field: getattr(p, field) for field in char_fields}

In [17]:
#| export

def set_charshape_pset(
    charshape_pset, value_dict:dict
):
    """
    CharShape값을 입력하기 위한 함수입니다.
    char_fields에 정의된 키 값을 기반으로 파라미터를 세팅합니다.
    """
    
    for field in char_fields:
        value = value_dict.get(field, None)
        if not value:
            continue
        setattr(charshape_pset, field, value)

    return charshape_pset

In [18]:
#| export
def get_parashape_pset(p):

    return {field: getattr(p, field) for field in para_fields}

In [19]:
#| export
def set_parashape_pset(
    parashape_pset, value_dict:dict,
):
 
    for field in para_fields:
        value = value_dict.get(field, None)
        if not value:
            continue
        setattr(parashape_pset, field, value)

    
    return parashape_pset

In [20]:
#| export


def hex_to_rgb(hex_string):
    # Remove the "#" symbol if it exists
    if hex_string.startswith("#"):
        hex_string = hex_string[1:]

    # Convert the hex string to decimal integers
    red = int(hex_string[0:2], 16)
    green = int(hex_string[2:4], 16)
    blue = int(hex_string[4:], 16)

    # Return the RGB tuple
    return (red, green, blue)


def get_rgb_tuple(color):
    # check if the input is already a tuple
    if isinstance(color, tuple):
        # validate each color
        if len(color) > 3:
            raise ValueError(
                f"colors should contains three compoents which represents RGB"
            )

        for component in color:
            if component > 255:
                raise ValueError(
                    f"number should be smaller than 255. the value is {color}"
                )
        return color

    # if the input is a string, split it into a list of colors
    elif isinstance(color, str):
        colors = {
            "red": (255, 0, 0),
            "green": (0, 255, 0),
            "blue": (0, 0, 255),
            "black": (0, 0, 0),
            "white": (255, 255, 255),
        }

        if color in colors.keys():
            return colors.get(color)

        # validate each color
        if not (
            color.startswith("#")
            and len(color) == 7
            and all(c in "0123456789abcdefABCDEF" for c in color[1:])
        ):
            raise ValueError(f"'{color}' is not a valid hexadecimal color.")

        # convert the list to a tuple and return it
        return hex_to_rgb(color)

    else:
        raise TypeError("Input must be a string or a tuple of colors.")

In [21]:
#| export 
def convert_to_hwp_color(color):
    
    if isinstance(color, int):
        return color 
    
    if isinstance(color, str):  # if the color is a string, we assume it's a hex string
        #hwp use bgr order
        colors = {
            "red": "0000FF",
            "green": "00FF00",
            "blue": "FF0000",
            "black": "000000",
            "white": "FFFFFF",
        }
            
        if color in colors.keys():
            return int(colors.get(color), 16)
        
        # handle hex
        m = re.search("^#?([0-9A-Fa-f]{6})$", color)
        if m:
            color = m.group(1)
            return int(f"{color[4:6]}{color[2:4]}{color[0:2]}", 16)
        
    elif type(color) == tuple and len(color) == 3:  # if the color is a tuple, we assume it's an (R,G,B) tuple
        return color[2]*65536 + color[1]*256 + color[0]
    else:
        raise ValueError(f"Unsupported color format: {color}")

In [22]:
#| export
def convert_hwp_color_to_hex(color:int):
    if not color:
        return color
    text = f"{color:06x}"
    return f"#{text[4:6]}{text[2:4]}{text[:2]}"


In [23]:
print(convert_to_hwp_color("00FFFF"))  # Outputs: 65535
print(convert_to_hwp_color((0, 255, 255)))  # Outputs: 65535

16776960
16776960


In [24]:
#| export 

def mili2unit(value):
    """
    1 밀리는 283 hwpunit 입니다.
    """
    return int(round(value*283, 0)) if value else value

In [25]:
#| export 

def unit2mili(value):
    return value/283 if value else value

In [26]:
#| export 
def point2unit(value):
    """
    1point는 100 hwpunit입니다.
    """
    return int(round(value*100, 0)) if value else value

In [27]:
#| export

def unit2point(value):
    return value / 100 if value else value

In [ ]:
#| export 

def to_hwpunit(value, unit):
    if unit == "mili":
        return mili2unit(value)
    if unit == "pt":
        return point2unit(value)
    
def from_hwpunit(value, unit):
    if unit == "mili":
        return unit2mili(value)
    if unit in ["pt", "point"]:
        return unit2point(value)

In [28]:
#| export 

def block_input(func):
    """
    함수가 실행될 동안 다른 입력을 할 수 없게 하는 기능을 가진 데코레이터입니다. 
    """
    def wrapper(app, *args, **kwargs):
        app.api.EditMode = 0
        result = func(app, *args, **kwargs)
        app.api.EditMode = 1
        return result
    return wrapper


In [29]:
#| hide
import nbdev

nbdev.nbdev_export()